# This notebook is for making multipanel plots of just two sims with slices

### For comparasion of the two sims, and eventually for making movies

You can change the field to be anything you want. You could also make it more than just two panels, if you wanted to include multiple fields, but I'd reccomend against, since it'll be crowded and a lot to look at.

In [ ]:
# code below is from Claire's git, which I've modified to fit my simulations

####################################################
# Plot multipanel figures comparing sim variants
####################################################

# imports
import yt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, SymLogNorm, LinearSegmentedColormap
from matplotlib.ticker import FixedLocator, MultipleLocator
from mpl_toolkits.axes_grid1 import ImageGrid

plt.rcParams.update({"font.size":18})

# gonna load this in the hard way since noramp is missing ds 34
snap_list = np.arange(0,82)
snap_list = np.delete(snap_list, 34)

for snap in snap_list:
    
    # load the data
    if snap < 10:
        file = '000'+str(snap)
        newCGM = yt.load("/mnt/research/galaxies-REU/sims/claire_isogal_sims/newCGM_h2sf/DD"+file+"/DD"+file)
        noramp = yt.load("/mnt/research/galaxies-REU/sims/claire_isogal_sims/newCGM_h2sf_no-ramp/DD"+file+"/DD"+file)

    if snap >= 10:
        file = '00'+str(snap)
        newCGM = yt.load("/mnt/research/galaxies-REU/sims/claire_isogal_sims/newCGM_h2sf/DD"+file+"/DD"+file)
        noramp = yt.load("/mnt/research/galaxies-REU/sims/claire_isogal_sims/newCGM_h2sf_no-ramp/DD"+file+"/DD"+file)
        
    # make the center and width of the plots
    center = 0.5
    width = yt.YTQuantity(412, 'kpc')
    extent = (-width/2, width/2, -width/2, width/2)

    field = ('gas', 'radial_velocity')
    field_name = 'radial_velocity'

    # make a function for getting the plots
    def prep_frb(ds):
        # uncomment line below if want only a data object instead of all data
        # mysphere = ds.sphere([center,center,center], width/2)

        s = yt.SlicePlot(ds, 'x', field, width=width)
                          
        frb = s.data_source.to_frb(width, 512)

        return frb


    # make frbs
    frb_newCGM = prep_frb(newCGM)
    frb_noramp = prep_frb(noramp)


    # make the figure and grid
    fig = plt.figure(figsize=(15,10))
    grid = ImageGrid(fig, 111, nrows_ncols=(1,2),
                    axes_pad=0, label_mode='1', share_all=True,
                    cbar_mode='edge', cbar_location='right',
                    cbar_pad=0)

    grid.axes_llc.tick_params(labelleft=False, labelbottom=False)
    for ax in grid:
        ax.tick_params(which='both', axis='both', direction='in')
        ax.xaxis.set_major_locator(FixedLocator([-300,-200,-100,0,100,200,300]))
        ax.xaxis.set_minor_locator(MultipleLocator(20))

    # make the limit
    v_norm = SymLogNorm(500, linscale=0.1, base=10, vmin=-5e2, vmax=5e2)

    # get time
    time = str(np.round(newCGM.current_time.to('Gyr'), 2))

    # start plotting
    ax = grid.axes_column[0]
    ax[0].text(0.04, 0.92, "NewCGM", 
               transform=ax[0].transAxes,
               fontdict={'size':'x-large','weight':'bold','color':'black'})
    ax[0].text(0.04, 0.05, time+' Gyr', 
               transform=ax[0].transAxes,
               fontdict={'size':'large','color':'black'})

    v_newCGM = ax[0].imshow(np.array(frb_newCGM['radial_velocity']/1e5), 
                            origin='lower', extent=extent,
                            cmap='seismic', norm=v_norm)

    # next sim
    ax = grid.axes_column[1]
    ax[0].text(0.04, 0.92, "NoRamp", 
               transform=ax[0].transAxes,
               fontdict={'size':'x-large','weight':'bold','color':'black'})

    v_noramp = ax[0].imshow(np.array(frb_noramp['radial_velocity']/1e5), 
                            origin='lower', extent=extent,
                            cmap='seismic', norm=v_norm)

    # add the colorbar
    v_cb = fig.colorbar(v_newCGM, cax=grid.cbar_axes[0], extend='both')

    # make the ticks on the colorbars where I want them
    v_cb.set_ticks(FixedLocator([-500,-100,-50,0,50,100,500]))
    v_cb.minorticks_off()

    # label the colorbar
    # This can get crowded, so use small/short labels
    v_cb.set_label(r'Radial Velocity [km/s]')

    # save the plot
    fig.savefig(field_name+"_ComparasionPlots_"+file+".png", transparent=True)